<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Gender%20Detection/GenderClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

100% 1.31G/1.32G [00:23<00:00, 66.7MB/s]
100% 1.32G/1.32G [00:23<00:00, 60.5MB/s]


In [ ]:
!unzip --qq gender-recognition-200k-images-celeba.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

#Hyper Parameter

In [ ]:
width = 178
height = 218
batch_size = 32

#Data Set


In [ ]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
)

train_data = data_generator.flow_from_directory(
    '/content/Dataset/Train',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    '/content/Dataset/Validation',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

test_data = data_generator.flow_from_directory(
    '/content/Dataset/Test',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
)

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
Found 20001 images belonging to 2 classes.


In [ ]:
test_data = data_generator.flow_from_directory(
    '/content/Dataset/Test',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
)

Found 20001 images belonging to 2 classes.


In [ ]:
ResNet = tf.keras.applications.ResNet50V2(include_top=False, pooling='avg', weights='imagenet',
input_shape=(178, 218, 3))

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
model = tf.keras.Sequential([
        ResNet,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(2, activation='sigmoid'),              
])

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=2667,
    validation_data=val_data,
    validation_steps=667,
 )

Epoch 1/5
2667/2667 [==============================] - 1619s 592ms/step - loss: 0.1345 - accuracy: 0.9477 - val_loss: 0.1408 - val_accuracy: 0.9463
Epoch 2/5
2667/2667 [==============================] - 1573s 590ms/step - loss: 0.0920 - accuracy: 0.9653 - val_loss: 0.0913 - val_accuracy: 0.9665
Epoch 3/5
2667/2667 [==============================] - 1585s 594ms/step - loss: 0.0776 - accuracy: 0.9713 - val_loss: 0.1228 - val_accuracy: 0.9568
Epoch 4/5
2667/2667 [==============================] - 1590s 596ms/step - loss: 0.0671 - accuracy: 0.9756 - val_loss: 0.0915 - val_accuracy: 0.9690
Epoch 5/5
2667/2667 [==============================] - 1589s 596ms/step - loss: 0.0616 - accuracy: 0.9777 - val_loss: 0.1928 - val_accuracy: 0.9303


In [ ]:
model.evaluate(test_data)

626/626 [==============================] - 110s 175ms/step - loss: 0.1821 - accuracy: 0.9314


[0.18208661675453186, 0.9314034581184387]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save('Gender2.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[11470    72]
 [ 1284  7175]]
Classification Report
              precision    recall  f1-score   support

      Female       0.90      0.99      0.94     11542
        Male       0.99      0.85      0.91      8459

    accuracy                           0.93     20001
   macro avg       0.94      0.92      0.93     20001
weighted avg       0.94      0.93      0.93     20001



# Telegram Bot Implementation

In [ ]:
!pip install retina-face
!pip install telebot

In [ ]:
import cv2
from tensorflow.python.keras.models import load_model
import telebot
from telebot import types
from retinaface import RetinaFace

In [ ]:
bot = telebot.TeleBot('Insert Your Token')


@bot.message_handler(content_types=['photo'])
def photo(message):
    print('message.photo =', message.photo)
    fileID = message.photo[-1].file_id
    print('fileID =', fileID)
    file_info = bot.get_file(fileID)
    print('file.file_path =', file_info.file_path)
    downloaded_file = bot.download_file(file_info.file_path)

    with open(f"BotPhotos/{fileID}.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    model = load_model('Gender2.h5')

    image = cv2.imread(f"BotPhotos/{fileID}.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = RetinaFace.extract_faces(image, align=True)

    for face in faces:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        image2 = cv2.resize(face, (178, 218))
        image2 = image2 / 255
        image2 = image2.reshape(1, 178, 218, 3)

        pred = model.predict([image2])

        result = np.argmax(pred)

        if result == 1:
            bot.reply_to(message, 'Female')

        elif result == 0:
            bot.reply_to(message, 'Male')


@bot.message_handler(commands=['start'])
def say_hello(message):
    bot.send_message(message.chat.id, f'wellcome Dear {message.from_user.first_name}')


@bot.message_handler(func=lambda message: True)
def send_unknown(message):
    bot.reply_to(message, 'I do not understand what you are saying')


bot.polling()